In [ ]:
from glob import glob
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.preprocessing import StandardScaler

In [ ]:
def process_file(path, path_total):
    total = pd.read_csv(path_total, dtype={"nadzsjd": str}, index_col=0).set_index(
        "nadzsjd"
    )
    data = gpd.read_parquet(path).set_index("nadzsjd")

    data_total = data.join(total)

    data_relative = data_total.drop(data.columns[:12], axis=1)

    if (
        path
        == "/data/uscuni-restricted/04_spatial_census/nadzsjd_households_2021.parquet"
    ):
        data_relative = data_relative.replace("d", np.nan).dropna(axis=0)
        data_relative[data_relative.columns.drop("geometry")] = data_relative[
            data_relative.columns.drop("geometry")
        ].astype(float)
    elif (
        path
        == "/data/uscuni-restricted/04_spatial_census/nadzsjd_housing_size_facilities_2021.parquet"
    ):
        data_relative = data_relative.drop(
            columns=[
                "Průměrná plocha 1 obydleného bytu v m2 v domech celkem",
                "Počet obytných místností(4 m2 a více) obydlených bytů v domech celkem",
                "Celková plocha obydlených bytů v m2 v domech celkem",
            ]
        )
    elif (
        path
        == "/data/uscuni-restricted/04_spatial_census/nadzsjd_housing_flats_2021.parquet"
    ):
        data_relative = data_relative.drop(columns="Neobydlené byty celkem")

    cols_to_normalize = data_relative.columns.drop(["Obyvatelstvo celkem", "geometry"])
    data_relative[cols_to_normalize] = data_relative[cols_to_normalize].div(
        data_relative["Obyvatelstvo celkem"], axis=0
    )

    data_relative = data_relative.dropna(axis=0)

    return data_relative

In [ ]:
path_total = "/data/uscuni-restricted/04_spatial_census/total.csv"

In [ ]:
files = [
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_pop_age_gender_2021.parquet",
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_pop_ea_gender_2021.parquet",
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_housing_flats_2021.parquet",
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_pop_residence_gender_2021.parquet",
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_emp_type_age_2021.parquet",
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_education_2021.parquet",
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_pop_status_gender_2021.parquet",
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_housing_houses_2021.parquet",
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_emp_ea_age_2021.parquet",
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_emp_employed_2021.parquet",
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_households_2021.parquet",
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_pop_nationality_2021.parquet",
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_pop_religion_gender_2021.parquet",
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_housing_size_facilities_2021.parquet",
]

In [ ]:
dfs = {}
for file in files:
    stem = Path(file).stem
    data_relative = process_file(file, path_total)
    dfs[stem] = data_relative

In [ ]:
# Iterate over the processed data in dfs
for i, df in dfs.items():
    pca = PCA()
    pca.fit(df.iloc[:, :-2])

    # Calculate cumulative explained variance
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

    # Find the number of components that explain at least 85% of the variance
    n_components_85 = np.argmax(cumulative_variance >= 0.85) + 1

    # Fit PCA again with the optimal number of components
    pca = PCA(n_components=n_components_85)
    pca.fit(df.iloc[:, :-2])

    # Compute Loadings
    loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

    # Convert to DataFrame for readability
    loadings_df = pd.DataFrame(
        loadings.T,
        index=[f"PC{k + 1}" for k in range(pca.explained_variance_.shape[0])],
        columns=df.iloc[:, :-2].columns,
    )

    # Transform and build DataFrame
    transformed = pca.transform(df.iloc[:, :-2])
    df_pca = pd.DataFrame(
        transformed,
        index=df.iloc[:, :-2].index,
        columns=[f"{i}_PC{k + 1}" for k in range(transformed.shape[1])],
    ).set_geometry(df.geometry)

    # Save to Parquet
    df_pca.to_parquet(f"/data/uscuni-restricted/05_pcs/{i}.parquet")

In [ ]:
all_files = glob("/data/uscuni-restricted/05_pcs/*.parquet")

In [ ]:
all_files = [
    "/data/uscuni-restricted/05_pcs/nadzsjd_pop_age_gender_2021.parquet",
    "/data/uscuni-restricted/05_pcs/nadzsjd_pop_ea_gender_2021.parquet",
    "/data/uscuni-restricted/05_pcs/nadzsjd_housing_flats_2021.parquet",
    "/data/uscuni-restricted/05_pcs/nadzsjd_pop_residence_gender_2021.parquet",
    "/data/uscuni-restricted/05_pcs/nadzsjd_emp_type_age_2021.parquet",
    "/data/uscuni-restricted/05_pcs/nadzsjd_education_2021.parquet",
    "/data/uscuni-restricted/05_pcs/nadzsjd_pop_status_gender_2021.parquet",
    "/data/uscuni-restricted/05_pcs/nadzsjd_housing_houses_2021.parquet",
    "/data/uscuni-restricted/05_pcs/nadzsjd_emp_ea_age_2021.parquet",
    "/data/uscuni-restricted/05_pcs/nadzsjd_emp_employed_2021.parquet",
    "/data/uscuni-restricted/05_pcs/nadzsjd_pop_nationality_2021.parquet",
    "/data/uscuni-restricted/05_pcs/nadzsjd_households_2021.parquet",
    "/data/uscuni-restricted/05_pcs/nadzsjd_pop_religion_gender_2021.parquet",
    "/data/uscuni-restricted/05_pcs/nadzsjd_housing_size_facilities_2021.parquet",
]

In [ ]:
dfs = []

# Process all files
for file in all_files:
    dfs.append(gpd.read_parquet(file))

In [ ]:
dfs = [df.set_geometry(df.geometry) for df in dfs]

# Drop additional geometry columns before concatenation
for i in range(1, len(dfs)):
    dfs[i] = dfs[i].drop(columns=["geometry"])

# Concatenate along columns
concat_df = pd.concat(dfs, axis=1)

# Reassign geometry column after concatenation
concat_df = gpd.GeoDataFrame(concat_df, geometry=dfs[0].geometry)

In [ ]:
# Concatenate all datasets
concat_df = pd.concat(dfs, axis=1)

In [ ]:
concat_df.to_parquet("/data/uscuni-restricted/05_pcs/merged_pcs.parquet")